In [1]:
import pandas as pd
import numpy as np


In [52]:
df = pd.read_csv('out.csv', usecols=['id','title','artist','year','genres'])
df.id = df.id.drop_duplicates()
df = df[np.isfinite(df['id']) == True]
df = df[np.isfinite(df['year']) == True]
df.year = df.year.astype(int)
df = df[df['title'].astype(str) != " "]
df = df[df['artist'].astype(str) != " "]
df = df[(df['year'] >= 1999 ) == True]
df = pd.concat([df[col].astype(str).str.lower() for col in df.columns], axis=1)
df.to_csv('discog.csv', index=False, encoding = 'utf-8')

In [55]:
df = pd.read_csv('pitchfork_reviews.csv', usecols =['reviewid', 'title','artist','year','label','score','best_new_music'])
df.reviewid = df.reviewid.drop_duplicates()
df = df[np.isfinite(df['reviewid']) == True]
df = pd.concat([df[col].astype(str).str.lower() for col in df.columns], axis=1)
df.to_csv('pitchfork.csv', index=False, encoding = 'utf-8')

In [57]:
import py_entitymatching as em
A = em.read_csv_metadata('pitchfork.csv', key='reviewid')
B = em.read_csv_metadata('discog.csv', key='id')

In [58]:
ob = em.OverlapBlocker()

In [66]:
C = ob.block_tables(A, B, 'title', 'title', word_level=True, overlap_size=3, 
                    l_output_attrs=['title', 'artist', 'year','label','score','best_new_music'], 
                    r_output_attrs=['title', 'artist', 'year','genres'],
                    show_progress=False)

In [67]:
ab = em.AttrEquivalenceBlocker()
D = ab.block_candset(C, 'title', 'title', show_progress=False)
D.to_csv('new_tuples_after_blocking.csv', index = False, encoding = 'utf-8')

In [119]:
df = pd.read_csv('possible_matches.csv', usecols = ['_id','ltable_reviewid','rtable_id','ltable_artist','ltable_title','ltable_year','rtable_title','rtable_artist','rtable_year','predicted'])
df = df[df['predicted'] == 1]
df = df.drop('predicted',1)
df = df.rename(columns = {'ltable_reviewid':'p_id','rtable_id': 'd_id','rtable_artist' : 'd_artist', 'ltable_artist' : 'p_artist','ltable_title': 'p_title',
                     'ltable_year': 'p_year','rtable_title': 'd_title','rtable_year': 'd_year'})
df.to_csv('matches.csv', index = False, encoding = 'utf-8')

In [80]:
p = pd.read_csv('new_tuples_after_blocking.csv')

In [111]:
p['predicted'] = df['predicted']
p = p[np.isfinite(p['predicted']) == True]
p = p.rename(columns={'ltable_reviewid':'p_id','rtable_id': 'd_id','ltable_label': 'label','ltable_score': 'score', 'ltable_best_new_music': 'best_new_music', 'rtable_artist' : 'artist',
                     'rtable_title': 'title','rtable_year': 'year', 'rtable_genres':'genres'})
p = p[['p_id','d_id','artist', 'title', 'label', 'year', 'genres','score','best_new_music']]
p.p_id = p.p_id.astype(int)
p.d_id = p.d_id.astype(int)
p.to_csv('table_E.csv', index = False, encoding = 'utf-8')